# Inference a fine-tuned model with function calling - hallucination scenario

## Objective

In this notebook, we'll demonstrate how to inference our fine-tuned model with function calling for the token reduction scenario. 

After your fine-tuned model is deployed, you can use it like any other deployed model via the chat completion API. 

## Time

You should expect to sepnd 5-10 min running this sample.

### Before you begin
#### Installation
The following packages are required to execute this notebook.

In [ ]:
# Install the packages\
%pip install requests openai~=1.10

Import the required packages.

In [ ]:
import json
from openai import AzureOpenAI

### Parameters

In [ ]:
azure_endpoint = "https://<YOUR_RESOURCE_NAME>.openai.azure.com"
api_version = "2024-02-15-preview"
aoai_api_key = "<AOAI_RESOURCE_API_KEY>"

## Run this example

Define gpt_test function for inference and initialize an Azure OpenAI client for interaction with the Azure OpenAI service.

In [ ]:
def gpt_test() -> None:
    print("gpt_inference")


client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=aoai_api_key,
    api_version=api_version,
)

Lets declare function schema. This schema can contain multiple functions which can perform multiple intents. Our stock use case features two functions: the first one retrieves the current stock price, and the second one gets the stock price of last n days. 

In [ ]:
functions = [
    {
        "name": "get_current_stock_price",
        "description": "Get the current stock price",
        "parameters": {
            "type": "object",
            "properties": {"symbol": {"type": "string", "description": "The stock symbol"}},
            "required": ["symbol"],
        },
    },
    {
        "name": "get_last_nday_stock_price",
        "description": "Get stock price last n days",
        "parameters": {
            "type": "object",
            "properties": {
                "symbol": {"type": "string", "description": "The stock symbol"},
                "period": {"type": "string", "description": "Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max"},
            },
            "required": ["symbol", "period"],
        },
    },
]

lets add the content of our test dataset (stock-test-hallucination.jsonl) into a list of strings.

In [ ]:
from pathlib import Path

# Replace 'Data/stock-test-token-reduction.jsonl' with the actual path to your file
file_path = Path("Data/stock-test-hallucination.jsonl")

with file_path.open(errors="ignore") as json_file:
    json_list = list(json_file)

We need to define the message with the question that we want to ask to gpt.

This code extracts system and user content from the JSON string, creates a list of messages to be sent to the Azure OpenAI model for completion, sends messages for completion and finally prints the model's completion response along with any errors encountered.

we set tempretre as 0 to reduce randomness.

In [ ]:
mismatch_count = 0
for i, json_str in enumerate(json_list[:1]):
    print("starting on ", i)
    result = json.loads(json_str)
    if len(result["messages"]) > 2:
        system_content = result["messages"][0]["content"]
        user_content = result["messages"][1]["content"]
    else:
        user_content = result["messages"][0]["content"]

    messages = [
        {"role": "system", "content": system_content},
        # {"role": "user", "content": user_content},
        {"role": "user", "content": "What was the closing price of Titan Robotics' stock last Friday"},  # fake company
        # {"role": "user", "content": "What was the highest price that walmart's stock reached last quarter?"}, # real company
    ]

    try:
        completion = client.chat.completions.create(
            model="<DEPLOYMENT_NAME>",  # add the fine-tuned model deployment name
            messages=messages,
            temperature=0.0,  # to reduce randomness
            functions=functions,
            function_call="auto",
        )
        try:
            response_message = completion.choices[0].message
            print(completion.choices[0].message.model_dump_json(indent=2))

        except Exception as e:
            print("Error", i, completion)
            print(e)

    except Exception as e:
        print("Error", i)
        print(e)


if __name__ == "__main__":
    gpt_test()

Let's now check the entire test dataset.

In [ ]:
# Initialize a list to store results for all examples
all_results = []

for i, json_str in enumerate(json_list):
    print("Processing example ", i)
    result = json.loads(json_str)
    if len(result["messages"]) > 2:
        system_content = result["messages"][0]["content"]
        user_content = result["messages"][1]["content"]
    else:
        user_content = result["messages"][0]["content"]

    # Construct user message dynamically from the dataset
    user_messages = [
        {"role": "system", "content": system_content},
    ]
    for message in result["messages"]:
        if message["role"] == "user":
            user_messages.append({"role": "user", "content": message["content"]})

    try:
        completion = client.chat.completions.create(
            model="<DEPLOYMENT_NAME>",  # add the fine-tuned model deployment name
            messages=user_messages,
            temperature=0.0,  # to reduce randomness
            functions=functions,
            function_call="auto",
        )
        try:
            response_message = completion.choices[0].message
            # Store the result for this example
            all_results.append(response_message.model_dump_json(indent=2))

        except Exception as e:
            print("Error", i, completion)
            print(e)

    except Exception as e:
        print("Error", i)
        print(e)

# Now you have results for all examples in `all_results`
# You can save this to a file or process it further as needed

if __name__ == "__main__":
    gpt_test()

In [ ]:
all_results